In [2]:
import numpy as np
import tifffile
from scipy import ndimage
from skimage.filters import threshold_otsu, gaussian
from skimage.morphology import binary_closing, disk
import os
import glob
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/flow3_1.4Pa_18h/Cadherins'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3_1.4Pa_18h/Membrane'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Find all Cadherin .tif files
cadherin_files = glob.glob(os.path.join(input_dir, '*Cadherins*.tif'))
print(f"Found {len(cadherin_files)} Cadherin files to process")

# Process each file
for cadherin_file in cadherin_files:
    # Get base filename for output
    filename = os.path.basename(cadherin_file)
    base_name = filename.replace('_Cadherins.tif', '')

    print(f"Processing: {filename}")

    # Load the image
    cadherin_img = tifffile.imread(cadherin_file)
    print(f"  Image shape: {cadherin_img.shape}")

    # Extract Cadherin channel if needed
    if len(cadherin_img.shape) == 2:
        # Single channel image (already Cadherin)
        print("  Detected single-channel Cadherin image")
        membrane_channel = cadherin_img
    elif len(cadherin_img.shape) == 3 and cadherin_img.shape[0] == 3:
        # Format is (C, H, W)
        print("  Detected format: (C, H, W)")
        membrane_channel = cadherin_img[0]  # First channel
    elif len(cadherin_img.shape) == 3 and cadherin_img.shape[2] == 3:
        # Format is (H, W, C)
        print("  Detected format: (H, W, C)")
        membrane_channel = cadherin_img[:, :, 0]  # First channel
    else:
        print(f"  Unexpected image shape: {cadherin_img.shape}. Using first channel/plane.")
        if len(cadherin_img.shape) == 3:
            membrane_channel = cadherin_img[0] if cadherin_img.shape[0] < cadherin_img.shape[1] else cadherin_img[:, :, 0]
        else:
            membrane_channel = cadherin_img

    # Apply Gaussian blur to reduce noise
    print("  Applying Gaussian blur...")
    membrane_smoothed = gaussian(membrane_channel, sigma=1)

    # Apply Otsu thresholding
    print("  Performing Otsu thresholding...")
    thresh = threshold_otsu(membrane_smoothed)
    binary_membrane = membrane_smoothed > thresh

    # Apply binary closing with larger radius to fill holes
    print("  Applying binary closing...")
    selem_close = disk(3)  # Increased radius to fill more holes
    closed_membrane = binary_closing(binary_membrane, selem_close)

    # Apply erosion to remove small isolated elements
    print("  Applying erosion to remove small elements...")
    selem_erode = disk(1)  # Small radius for erosion
    final_membrane = ndimage.binary_erosion(closed_membrane, structure=selem_erode)

    # Save only the membrane mask
    membrane_mask_path = os.path.join(output_dir, f"{base_name}_membrane_mask.tif")
    tifffile.imwrite(membrane_mask_path, final_membrane.astype(np.uint8))
    print(f"  Saved membrane mask to {membrane_mask_path}")

    # Print statistics
    print(f"  Threshold value used: {thresh}")
    print(f"  Percentage of pixels identified as membrane: {(final_membrane.sum() / final_membrane.size * 100):.2f}%")
    print(f"  Processing complete for {filename}\n")

print("All processing complete!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 8 Cadherin files to process
Processing: denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001_Cadherins_focus_bg_tophat.tif
  Image shape: (1024, 1024)
  Detected single-channel Cadherin image
  Applying Gaussian blur...
  Performing Otsu thresholding...
  Applying binary closing...
  Applying erosion to remove small elements...
  Saved membrane mask to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3_1.4Pa_18h/Membrane/denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001_Cadherins_focus_bg_tophat.tif_membrane_mask.tif
  Threshold value used: 0.36076910002060664
  Percentage of pixels identified as membrane: 28.23%
  Processing complete for denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001_Cadherins_focus_bg_tophat.tif

Processing: denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq002_Cadherins_focus_bg_tophat.tif
  Image shape: (1024, 1024)
  Detected single-cha

In [ ]:
# Import necessary libraries
from google.colab import drive
drive.mount('/content/drive')

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import tifffile
from pathlib import Path

# Set up paths
base_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3_1.4Pa_18h"
output_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3_1.4Pa_18h/Membrane_Adjusted"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Define function to extract sample ID from filenames
def extract_sample_id(filename):
    """
    Extract the sample ID from filenames like:
    - denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq005_Cadherins_focus_bg_tophat.tif_membrane_mask
    - denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq005_Nuclei_contrast_bg_tophat_mask

    We want to extract everything before the part that identifies it as Nuclei or membrane
    """
    # Convert to lowercase for case-insensitive matching
    lowercase_filename = str(filename).lower()

    # Extract the base sample ID - everything up to seq### part
    parts = str(filename).split('_')
    seq_part = None
    for i, part in enumerate(parts):
        if part.startswith('seq'):
            seq_part = i
            break

    if seq_part is not None:
        # Include the seq### part in the sample ID
        base_id = '_'.join(parts[:seq_part+1])
        return base_id

    # Fallback method if seq### pattern is not found
    return None

# Function to find matching files across different folders
def find_matching_files(base_dir):
    # Define component folders to look in (removed Golgi)
    component_folders = ['Nuclei', 'Membrane']

    # Dictionary to store file paths for each sample and component
    file_dict = {}

    # Scan through each component folder
    for component in component_folders:
        component_dir = os.path.join(base_dir, component)
        if not os.path.exists(component_dir):
            print(f"Warning: {component_dir} does not exist.")
            continue

        # Get all mask files in the component folder (may end with .tif, _mask, etc.)
        files = [f for f in os.listdir(component_dir) if
                any(ext in f.lower() for ext in ['.tif', 'mask'])]
        print(f"Found {len(files)} mask files in {component} folder.")

        for file in files:
            # Extract the sample ID from the filename
            sample_id = extract_sample_id(file)
            if sample_id:
                print(f"  Found {component} file: {file}")
                print(f"  Sample ID: {sample_id}")

                if sample_id not in file_dict:
                    file_dict[sample_id] = {}

                # Store the full path to the file
                component_key = component.lower()
                file_dict[sample_id][component_key] = os.path.join(component_dir, file)
            else:
                print(f"  Couldn't extract sample ID from: {file}")

    # Print all extracted sample IDs and their components
    print("\nExtracted sample IDs:")
    for sample_id, components in file_dict.items():
        print(f"  Sample: {sample_id}")
        for comp, path in components.items():
            print(f"    {comp}: {os.path.basename(path)}")

    # Find samples that have all required components (Membrane and Nuclei only)
    complete_samples = []
    for sample_id, components in file_dict.items():
        # Check if we have both membrane and nuclei data
        if 'membrane' in components and 'nuclei' in components:
            complete_samples.append(sample_id)
        else:
            missing = []
            if 'membrane' not in components:
                missing.append('membrane')
            if 'nuclei' not in components:
                missing.append('nuclei')
            print(f"  Sample {sample_id} is missing: {', '.join(missing)}")

    print(f"Found {len(complete_samples)} complete samples with membrane and nuclei data.")

    return file_dict, complete_samples

# Function to create adjusted membrane image (removing nuclei areas)
def create_adjusted_membrane(membrane_img, nuclei_img):
    """
    Create membrane image with pixels set to 0 where nuclei are present
    """
    # Convert to binary if not already
    membrane_binary = membrane_img > 0
    nuclei_binary = nuclei_img > 0

    # Create the mask where membrane does not overlap with nuclei
    membrane_without_nuclei = membrane_binary.copy()
    membrane_without_nuclei[nuclei_binary] = 0

    # Convert back to the original data type of membrane_img
    if membrane_img.dtype == bool:
        return membrane_without_nuclei
    else:
        return membrane_without_nuclei.astype(membrane_img.dtype) * (membrane_img.max() or 1)

# Function to process a single sample
def process_sample(sample_id, file_dict, output_dir):
    print(f"\nProcessing sample: {sample_id}")

    # Load images
    try:
        membrane_img = io.imread(file_dict[sample_id]['membrane'])
        nuclei_img = io.imread(file_dict[sample_id]['nuclei'])

        print(f"  Loaded membrane image: shape {membrane_img.shape}")
        print(f"  Loaded nuclei image: shape {nuclei_img.shape}")
    except Exception as e:
        print(f"  Error loading images for sample {sample_id}: {e}")
        return False

    # Create adjusted membrane image (removed golgi parameter)
    adjusted_membrane = create_adjusted_membrane(membrane_img, nuclei_img)

    # Create output filename, keeping the original filename pattern but adding "_adjusted"
    original_filename = os.path.basename(file_dict[sample_id]['membrane'])
    adjusted_filename = original_filename.replace('.tif', '_adjusted.tif')
    output_path = os.path.join(output_dir, adjusted_filename)

    # Save the adjusted membrane image
    try:
        tifffile.imwrite(output_path, adjusted_membrane)
        print(f"  Saved adjusted membrane to: {output_path}")
        return True
    except Exception as e:
        print(f"  Error saving adjusted membrane for sample {sample_id}: {e}")
        return False

# Find matching files
file_dict, complete_samples = find_matching_files(base_dir)

# Process all samples with complete data
successful_count = 0
failed_count = 0

print(f"\nProcessing {len(complete_samples)} samples...")

for i, sample_id in enumerate(complete_samples):
    print(f"\nSample {i+1}/{len(complete_samples)}: {sample_id}")
    result = process_sample(sample_id, file_dict, output_dir)
    if result:
        successful_count += 1
    else:
        failed_count += 1

print(f"\nProcessing complete!")
print(f"Successfully processed: {successful_count} samples")
print(f"Failed: {failed_count} samples")
print(f"Output directory: {output_dir}")

# Display a sample of the adjusted images for verification
if successful_count > 0:
    # Get a list of the adjusted membrane files
    adjusted_files = [f for f in os.listdir(output_dir) if f.endswith('_adjusted.tif')]

    if adjusted_files:
        # Display the first adjusted membrane image as an example
        sample_file = os.path.join(output_dir, adjusted_files[0])
        sample_img = io.imread(sample_file)

        plt.figure(figsize=(10, 10))
        plt.imshow(sample_img, cmap='gray')
        plt.title(f'Sample Adjusted Membrane: {adjusted_files[0]}')
        plt.axis('off')
        plt.show()

        print(f"Displayed sample adjusted membrane image: {adjusted_files[0]}")